In [87]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import linear_model
import matplotlib.pyplot as pyplot
import pickle
from matplotlib import style

data = pd.read_csv("student-mat.csv", sep=";")
print(data.head())

  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  ...  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher  ...   
1     GP   F   17       U     GT3       T     1     1  at_home     other  ...   
2     GP   F   15       U     LE3       T     1     1  at_home     other  ...   
3     GP   F   15       U     GT3       T     4     2   health  services  ...   
4     GP   F   16       U     GT3       T     3     3    other     other  ...   

  famrel freetime  goout  Dalc  Walc health absences  G1  G2  G3  
0      4        3      4     1     1      3        6   5   6   6  
1      5        3      3     1     1      3        4   5   5   6  
2      4        3      2     2     3      3       10   7   8  10  
3      3        2      2     1     1      5        2  15  14  15  
4      4        3      2     1     2      5        4   6  10  10  

[5 rows x 33 columns]


In [88]:
data = data[["G1", "G2", "G3", "studytime", "failures", "absences"]]
print(data.head())

   G1  G2  G3  studytime  failures  absences
0   5   6   6          2         0         6
1   5   5   6          2         0         4
2   7   8  10          2         3        10
3  15  14  15          3         0         2
4   6  10  10          2         0         4


# Linear Regression

In [89]:
predict = "G3"

X = np.array(data.drop([predict], axis=1))
y = np.array(data[predict])

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.1)


best = 0
for _ in range(30):

    x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.1)

    linear = linear_model.LinearRegression()

    linear.fit(x_train, y_train)

    acc = linear.score(x_test, y_test)
    print(acc)

    if acc > best:
        best = acc
        # saves model
        with open("studentmodel.pickle", "wb") as f:
            pickle.dump(linear, f)

0.8338616192073317
0.8632021578590856
0.8521882923575552
0.853265859756378
0.7408076789165681
0.908592121215727
0.7028435890467664
0.908416210753622
0.8906178168816279
0.7887311021999294
0.8516378081240088
0.7996117634827443
0.7694406315054976
0.8997531301414984
0.8209880115655617
0.869479233098628
0.7006135826296949
0.8318365236657221
0.8341686041735368
0.8243697898835528
0.7448133542599262
0.8569277306305846
0.7719128725626931
0.8959036777142901
0.8711186373656992
0.7801978518905918
0.8225055564143566
0.608228273409153
0.8964372899935895
0.8025231175863646


In [90]:
pickle_in = open("studentmodel.pickle", "rb")
linear = pickle.load(pickle_in)

In [91]:
print("Coefficient:", linear.coef_)
print("Intercept:", linear.intercept_)


Coefficient: [ 0.15087756  0.99090828 -0.16210898 -0.1999162   0.04490664]
Intercept: -1.7290313570563693


In [92]:
predictions = linear.predict(x_test)

for x in range(len(predictions)):
    print(predictions[x], x_test[x], y_test[x])

6.359568943797024 [8 7 2 0 6] 9
12.94220408005292 [13 13  3  0  7] 14
7.243146374077124 [8 8 1 0 0] 11
16.21039789458372 [14 16  1  0  3] 16
15.673482215107887 [16 15  2  0 10] 15
10.535143910625495 [10 11  2  0  4] 9
10.274767489484024 [12 10  2  1 18] 10
19.291057654124643 [17 18  2  0 21] 18
14.447565188876306 [13 15  4  0  0] 15
9.062853958286807 [ 8 10  2  0  0] 12
12.242390411413453 [13 11  3  1 40] 11
6.252238090565397 [8 7 1 0 0] 6
10.686021466357175 [11 11  2  0  4] 11
11.989916288414353 [12 12  1  0  4] 13
9.67759560829562 [11 10  1  0  0] 10
19.538847910827094 [18 18  1  1 24] 18
9.763842613191374 [ 8 10  1  0 12] 10
11.598347898269191 [10 12  1  0  2] 12
8.549951219875656 [ 7 10  3  1  0] 10
13.92226171691388 [11 12  2  0 54] 11
13.822819292321084 [13 13  2  0 23] 13
7.993967878949482 [10  8  1  0 10] 9
18.510127210779522 [17 18  1  0  0] 18
7.814341321585882 [10  8  1  0  6] 11
7.488920443894033 [5 9 2 2 6] 7
8.74418532804773 [9 9 1 0 8] 10
13.798392445541946 [13 14  3  0 